In [15]:
# Import packages
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from threading import Thread
from queue import Queue
import os
import re
import polars as pl
from googleapiclient.http import MediaIoBaseDownload
from io import BytesIO
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle


In [ ]:
pip install google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2

Check if a `token.pickle` file already exists before running the following code. If the file exists, it is recommended to delete it and regenerate one.

In [17]:
# Scopes for accessing Google Drive
SCOPES = ['https://www.googleapis.com/auth/drive']

# Authenticate and create the service object
def authenticate_drive_api():
    creds = None
    # Token file for saving the authentication
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no credentials, perform authentication
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secrets.json', SCOPES)  # Ensure 'credentials.json' is downloaded from Google API Console
            creds = flow.run_local_server(port=0)
        # Save the credentials for future use
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return build('drive', 'v3', credentials=creds)

# Initialize the service object
service = authenticate_drive_api()


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=389849867563-4uggnm57nqe52156v32gj1lkosoqpoem.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A37635%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=GyfvJGpr2mtpuRzI6AP8xi3h1k3QIH&access_type=offline


This function grabs the list of all files in a specified folder that are not trashed and stores them into a list. Each file has an ID and name attribute that we will use later.

In [27]:
def find_files(folder_id):
    # Query to find files in the specified folder
    query = f"'{folder_id}' in parents and trashed=false"
    files = []

    # List files in the folder and append to list
    page_token = None
    while True:
        response = service.files().list(
            q=query,
            spaces='drive',
            fields='nextPageToken, files(id, name)',
            pageToken=page_token
        ).execute()

        files += response.get('files', [])

        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break
    return files

Call the function to find files for the specified folder.

In [28]:
# Get the list of files in the folder
folder_id = '14idmMBbM5xXZg4b61iINHbBTl2z4yLeD'
files = find_files(folder_id)

In [ ]:
# Process each file to add cyclone id

# Directory to save the processed files locally
output_dir = "processed_files"
os.makedirs(output_dir, exist_ok=True)

# Process each file
for file in files:
    file_id = file['id']
    file_name = file['name']

    # Extract the cyclone ID and name from the filename
    cyclone_id = '_'.join(file_name.split('_')[:3])
    cyclone_name = file_name.split('_')[3]

    # Download the file content
    request = service.files().get_media(fileId=file_id)
    file_stream = BytesIO()
    downloader = MediaIoBaseDownload(file_stream, request)
    done = False
    while not done:
        status, done = downloader.next_chunk()
    file_stream.seek(0)
    content = file_stream.read().decode('utf-8')

    # Add the cyclone id and name as a new column using Polars

    df = pl.read_csv(BytesIO(content.encode('utf-8')),separator='\t', has_header=False)
    df = df.with_columns([
    pl.lit(cyclone_id).alias("cyclone_id"),
    pl.lit(cyclone_name).alias("cyclone_name")
    ])

    # Save the modified DataFrame locally
    output_file_path = os.path.join(output_dir, file_name)
    df.write_csv(output_file_path, separator='\t',include_header=False)

    print(f"Processed and saved: {output_file_path}")



In [37]:
import glob

# Directories for processed files and output
input_dir = "processed_files"
output_dir = "combined_files"
os.makedirs(output_dir, exist_ok=True)

# File patterns to combine
patterns = {
    "Reduced_Trackfile": os.path.join(input_dir, "*Reduced_Trackfile*.txt"),
    "WWLLN_Locations": os.path.join(input_dir, "*WWLLN_Locations*.txt")
}

# Combine files based on patterns
for pattern_name, pattern_path in patterns.items():
    combined_content = []
    output_file_path = os.path.join(output_dir, f"Combined_{pattern_name}.txt")

    # Find all matching files
    matching_files = glob.glob(pattern_path)
    print(f"Combining {len(matching_files)} files for pattern '{pattern_name}'...")

    with open(output_file_path, "w") as output_file:
        for file_path in matching_files:
            with open(file_path, "r") as input_file:
                for line in input_file:
                    output_file.write(line)

    print(f"Combined file saved: {output_file_path}")


Combining 992 files for pattern 'Reduced_Trackfile'...
Combined file saved: combined_files/Combined_Reduced_Trackfile.txt
Combining 994 files for pattern 'WWLLN_Locations'...
Combined file saved: combined_files/Combined_WWLLN_Locations.txt
